# tootltips

In [ ]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

# Imports

In [2]:
from elasticsearch import Elasticsearch

# Client Elasticsearch

In [3]:
es_client = Elasticsearch()

# Preparer la base de données pour l'analyse au moment de la sauvegarde

In [73]:
index_name = "dpe_pre_1948_td005"
index_configurations = {
    "settings": {
        "index": {
            "number_of_shards": 1,
            "number_of_replicas": 0,
            "max_result_window": 500000,

        }
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "text_to_analyze": {
                "type": "text",
                "analyzer": "french"      
            },
            "category_index":{
                'type':'keyword'
            }
        }
    }
}

In [18]:
es_client.indices.delete(
    index=index_name,
)

{'acknowledged': True}

In [19]:
es_client.indices.create(
    index=index_name,
    body=index_configurations
)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'dpe_pre_1948_td005'}

# load tables

In [8]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths


if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        print(dept_dir)
        annexe_dept_dir = annexe_dir / dept_dir.name
        annexe_dept_dir.mkdir(exist_ok=True, parents=True)
        # LOAD TABLES
        td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
        td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
        td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
        td006 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)

        break
        # ENVELOPPE PROCESSING


D:\data\dpe_full\depts\1


In [37]:
mixte = ('mixte', 'combine', 'chauffage + ecs','chauffage et ecs','lie ','combine ',"idem")
mixte = ('mixte', 'combine', 'chauffage ecs','lie ','combine ',"idem")


gen_ecs_normalized_lib_matching_dict_ft = {
    "ECS thermodynamique electrique(PAC ou ballon)": [
        ('pompe a chaleur', 'pac', 'thermodynamique', 'air extrait', 'air exterieur', 'air ambiant')],
    
    "ballon a accumulation electrique": [('ballon', 'classique', 'accumulation'), ('electricite', 'electrique','elec.')],
    "ecs electrique indeterminee": [('electricite', 'electrique','elec.')],
    "ecs instantanee electrique": ['instantanee', ('electricite', 'electrique','elec.')],

    'chaudiere mixte gaz': ["chaudiere", mixte, "gaz"],
    'chaudiere mixte fioul': ["chaudiere", mixte, "fioul"],
    'chaudiere mixte bois': [("bois", "biomasse")],

    'chauffe-eau gaz independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gaz"],
    'chauffe-eau gpl independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gpl"],

    'chauffe-eau fioul independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"),
                                      "fioul"],
    "ecs collective reseau chaleur": ["reseau", "chaleur"],

    'chaudiere gaz': ["chaudiere", "gaz"],
    'chaudiere gpl': ["chaudiere", "gpl"],

    'chaudiere fioul': ["chaudiere", "fioul"],
    'chaudiere non déterminee': ["chaudiere"],
    'production mixte indeterminee': [ mixte],

}

In [10]:
from generate_dpe_annexes_scripts.utils import strip_accents

In [42]:
td005

,id,td001_dpe_id,tr011_sous_categorie_fiche_technique_id,valeur_renseignee,est_efface
0,3390,178,1,01 Ain,0
1,3391,178,2,520,0
2,3392,178,3,Maison Individuelle,0
3,3393,178,4,2001 - 2005,0
4,3394,178,5,190,0
...,...,...,...,...,...
2100215,174105056,9431386,16,Panneaux rayonnants NFC (système individuel)<br/>,0
2100216,174105057,9431386,17,Chauffe-eau électrique (système individuel)<br...,0
2100217,174105058,9431386,13,Porte(s) bois avec double vitrage<br/>Surface ...,0
2100218,174105059,9431386,12,Portes-fenêtres battantes PVC double vitrage 1...,0


In [11]:
td005_ecs = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '17']
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# feed es

In [20]:
from elasticsearch.helpers import bulk,parallel_bulk

In [21]:
# TODO : optimize

In [22]:
def gendata():
    for id_,vr in zip(td005_ecs.td001_dpe_id,td005_ecs.valeur_renseignee):
        yield {
            "_index": index_name,
    "id": id_,
    "text_to_analyze": vr.split('\n')
        }

bulk(es_client, gendata())

(92242, [])

# catégoriser mes documents

In [38]:
char_list=gen_ecs_normalized_lib_matching_dict_ft['chaudiere mixte gaz']
new_char_list=list()
for char in char_list:
    if isinstance(char,str):
        new_char_list.append(char)
    if isinstance(char,tuple):
        new_char_list.append('('+' OR '.join(char)+')')
search_instruction = ' AND '.join(new_char_list)
search_instruction

'chaudiere AND (mixte OR combine OR chauffage ecs OR lie  OR combine  OR idem) AND gaz'

In [48]:
search_body = {
    "query": {
        "query_string": {
            "query": search_instruction,
            "default_field": "text_to_analyze"
        },

    },

}

es_client.count(index=index_name, body=search_body)

{'count': 22049,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [54]:
es_client.search?

In [58]:
a_dict=es_client.search(index=index_name, body=search_body,size=500000)

In [66]:
hits = a_dict['hits']['hits']

In [71]:
s=pd.Series(index=[el['_source']['id'] for el in hits])
s[:]='chaudiere mixte gaz'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [72]:
s

1092714    chaudiere mixte gaz
4758732    chaudiere mixte gaz
262543     chaudiere mixte gaz
2044262    chaudiere mixte gaz
2754776    chaudiere mixte gaz
                  ...         
371016     chaudiere mixte gaz
2317526    chaudiere mixte gaz
1305371    chaudiere mixte gaz
923505     chaudiere mixte gaz
1777301    chaudiere mixte gaz
Length: 22049, dtype: object

# Sauvegarder un document

In [6]:
document = {
    "id": "some_id-1",
    "text_to_analyze": [
        "pompe à chaleur pac geothermique"
    ]
}

es_client.index(index=index_name, body=document)

# Recherche

In [55]:
search_body = {
    "query": {
        "query_string": {
            "query": '"pompe chaleur"~3 AND *therm*',
            "default_field": "text_to_analyze"
        }
    }
}

es_client.search(index=index_name, body=search_body)